In [2]:
#%%
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torch
from torch.utils.data import Subset
from torchsummary import summary
import numpy as np

In [3]:
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#----1.数据:Fashion-MNIST 3x299x299,仅取n%----
transform = transforms.Compose([
    transforms.Resize(299), # 缩放图像到299x299（Inception-v3要求的输入尺寸）
    transforms.Grayscale(num_output_channels=3),# 灰度图转3通道（Inception-v3要求3通道输入）
    transforms.ToTensor()  # 转换为Tensor：将PIL图像转为[0,1]范围的张量，维度为(C,H,W)
])
train_full = datasets.FashionMNIST('data', train=True, download=True, transform=transform)
test_full = datasets.FashionMNIST('data', train=False, download=True, transform=transform)

n = 100  # 根据硬件实际情况选取不同比例的数据
rng = np.random.default_rng(42) # 固定随机种子42
train_idx = rng.choice(len(train_full), len(train_full) // n, replace=False)
test_idx = rng.choice(len(test_full), len(test_full) // n, replace=False)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(Subset(train_full, train_idx), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(Subset(test_full, test_idx), batch_size=batch_size, shuffle=False)

100%|██████████| 26.4M/26.4M [00:05<00:00, 4.53MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 164kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.71MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 14.5MB/s]


In [5]:
#----2.导入官方Inception-v3和权重,只换fc----
model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 10)# 替换全连接层
model = model.to(device)

# 定义优化器：Adam优化器，学习率1e-4（微调预训练模型用小学习率），weight_decay=1e-5（L2正则化，防止过拟合）
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# summary(model, input_size=(3, 299, 299))   #打印模型

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:21<00:00, 5.09MB/s] 


In [6]:
#----3.训练/测试----
accs, losses = [], []
epochs = 10
for epoch in range(epochs):
    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        out = model(x).logits
        
        loss = F.cross_entropy(out, y)
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        correct, total_loss = 0, 0.
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            total_loss += F.cross_entropy(out, y).item()
            correct += (out.argmax(1) == y).sum().item()
    
    acc = correct / len(test_loader.dataset)
    avg_loss = total_loss / len(test_loader)
    accs.append(acc)
    losses.append(avg_loss)
    
    print(f'epoch {epoch}: loss={avg_loss:.4f}, acc={acc:.4f}')
#%%

epoch 0: loss=1.0314, acc=0.6600
epoch 1: loss=0.5821, acc=0.7700
epoch 2: loss=0.5644, acc=0.7800
epoch 3: loss=0.5485, acc=0.7900
epoch 4: loss=0.5381, acc=0.7800
epoch 5: loss=0.5444, acc=0.7800
epoch 6: loss=0.5964, acc=0.7600
epoch 7: loss=0.5293, acc=0.8000
epoch 8: loss=0.7567, acc=0.7300
epoch 9: loss=0.5786, acc=0.7800
